In [2]:
%load_ext autoreload
%load_ext dotenv
%autoreload 2
%dotenv

In [5]:
from pathlib import Path
import torch
import torchvision.transforms.v2 as t
from lightning import Trainer
from segmentation_models_pytorch import Unet

import sys; sys.path.append("../") if "../" not in sys.path else None
from datasets.datamodules import ImageDatasetDataModule 
from datasets.inria import InriaHDF5 
from training.tasks import ClassificationTask
from training.utils import (
    setup_logger, setup_wandb_logger, setup_checkpoint, #setup_eval
)
from training.evaluation import EvaluationReport, EvaluationMetricsLogger
from training.inference import InferenceCallback
from training.attribution import Attribution 
from etl.pathfactory import PathFactory
from etl.etl import reset_dir

import os, logging

from lightning.pytorch.utilities import disable_possible_user_warnings # type: ignore
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
disable_possible_user_warnings()

os.environ["WANDB_CONSOLE"] = "off"
os.environ["WANDB_SILENT"] = "true"

In [6]:
DATASET = InriaHDF5 
# MODEL = Unet
experiment = {
    "name": "testing_new_callbacks",
    "model_name": "unet",
    "model_params": {
        "encoder": "resnet18",
        "decoder": "deconvolution",
        "weights": "imagenet",
    },

    "dataset_name": DATASET.NAME,
    "task": DATASET.TASK,
    "num_classes": DATASET.NUM_CLASSES,
    "class_names": DATASET.CLASS_NAMES,

    "random_seed": 69,

    "test_split": 0.2,
    "val_split": 0.1,
    "batch_size": 2,
    "grad_accum": 1,
    "num_workers": 4,

    "loss": "binary_cross_entropy",
    "loss_params": {
        "reduction": "mean",
    },

    "optimizer": "adam",
    "optimizer_params": {
        "lr": 1e-5,
    },

    "monitor_metric": "iou",
    "monitor_mode": "max",

    "tile_size": (512, 512),
    "tile_stride": (512, 512),
}
PATHS = PathFactory(experiment["dataset_name"], experiment["task"])
LOGS_DIR = PATHS.experiments_path / experiment["name"]

# NOTE: t.Normalize(DATASET.MEANS, DATASET.STD_DEVS),
image_transform = t.Compose([t.ToImage(), t.ToDtype(torch.float32, scale=True)])
mask_transform = t.Compose([t.ToImage(), t.ToDtype(torch.float32, scale=False)])
#augmentations = t.Compose([t.Pad(6)])
augmentations = None

datamodule = ImageDatasetDataModule(
    root = PATHS.path / "inria.h5",
    is_remote = False,
    is_streaming = False,
    dataset_constructor = DATASET, 
    # dataframe = dataset_df,
    image_transform = image_transform,
    target_transform = mask_transform,
    common_transform = augmentations,
    **experiment
)
model = Unet(experiment["model_params"]["encoder"], classes=experiment["num_classes"]) 
display(datamodule)

logger = setup_logger(
    logs_dir = PATHS.experiments_path, 
    name = experiment["name"]
)

wandb_logger = setup_wandb_logger(
    logs_dir = PATHS.experiments_path,
    name = experiment["name"]
)

checkpoint_callback = setup_checkpoint(
    ckpt_dir = Path(logger.log_dir, "model_ckpts"),
    metric = experiment["monitor_metric"],
    mode = experiment["monitor_mode"],
    save_top_k = "all"
) 

# reset_dir(LOGS_DIR)


        
Local Dataset: urban_footprint @ [/home/sambhav/datasets/urban_footprint/inria.h5]
Configured For: segmentation
        

Local Logging To : /home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks
WandB Logging To: /home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks/wandb
Checkpoint Monitoring: val/iou, Checkpoints Saved To: /home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks/model_ckpts


In [ ]:
BEST_CKPT = checkpoint_callback.best_model_path 
LAST_CKPT = checkpoint_callback.last_model_path

trainer = Trainer(
    callbacks=[checkpoint_callback, EvaluationMetricsLogger()],
    #enable_checkpointing=False,
    logger = [logger],
    enable_model_summary=False,
    #fast_dev_run=True,
    #num_sanity_val_steps=0,
    max_epochs=12,
    check_val_every_n_epoch=3, 
    limit_val_batches=200,
    limit_train_batches=200,
    limit_test_batches=200,
)

In [ ]:

# experiment["optimizer_params"]["lr"] =  5e-6 
trainer.fit(
    model=ClassificationTask(model, **experiment),
    datamodule=datamodule,
    ckpt_path=LAST_CKPT if Path(LAST_CKPT).is_file() else None,
)

trainer.test(
    model=ClassificationTask(model, **experiment),
    datamodule=datamodule,
    ckpt_path=LAST_CKPT if Path(LAST_CKPT).is_file() else None,
    verbose=False
)

In [ ]:
EvaluationReport.plot_experiment_eval_report(
    logs_dir=LOGS_DIR,
    monitor_metric=experiment["monitor_metric"],
    save = True,
)

In [19]:
# LOGS_DIR -> inference -> epoch={epoch}_step={step} -> {split} -> inference_dataset.csv, austin1_0_512_0_512_footprints.geojson, ...
# LOGS_DIR -> attribution ->  epoch={epoch}_step={step} -> {filter_by}_{k} -> attribution_dataset.csv, austin1_0_512_0_512_iou={iou}_dice={dice}_gradcam.png, ...
epoch = 11 
step = 2400 
filter_by = "worst"
k = 25 

top_k_df = EvaluationReport.get_top_k_df(
    logs_dir = LOGS_DIR,
    epoch = epoch,
    step = step,
    split = "val",
    filter_by = filter_by,
    k = k
)

Loading Dataset From: /home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks/dataset.csv
Loading Samples From: /home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks/eval/epoch=11_step=2400_val_samples.csv
worst split
Returning the worst-25 samples, by IoU


In [ ]:
inference_dataset = InriaHDF5(
    root = PATHS.path / "inria.h5",
    split = "val",
    df = top_k_df,
    image_transform=image_transform,
    target_transform=mask_transform,
    common_transform=None,
    **experiment,
)

inference_dataloader = torch.utils.data.DataLoader(
    dataset = inference_dataset,
    batch_size = experiment["batch_size"] // experiment["grad_accum"],
    shuffle = False,
)

inference_trainer = Trainer(
    callbacks=[InferenceCallback(LOGS_DIR, epoch, step)],
    #limit_predict_batches=5,
)

inference_trainer.predict(
    model = ClassificationTask(model, **experiment),
    dataloaders = inference_dataloader,
    ckpt_path = LOGS_DIR / "model_ckpts" / f"epoch={epoch}_step={step}.ckpt",
    return_predictions = False,
)

In [20]:
attribution_dataset = DATASET(
    root = PATHS.path / "inria.h5",
    split = "val",
    df = top_k_df,
    image_transform=image_transform,
    target_transform=mask_transform,
)

Attribution(LOGS_DIR, 11, 2400, model, attribution_dataset).compute()

val custom dataset @ [/home/sambhav/datasets/urban_footprint/inria.h5]
writing attributions @ [/home/sambhav/experiments/urban_footprint_segmentation/testing_new_callbacks/attribution/epoch=11_step=2400]


Saving Attribution Maps:   0%|          | 0/25 [00:00<?, ?it/s]